## Solveur SAT

In [932]:
MAX_GRADE = 21

In [933]:
size = 100
nb_grades = 5
noise = 0
nb_class = 1

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()

gen.analyze_gen()


if nb_class == 1:
    admissions = admissions.astype(int)


---------Analyze----------
Lambda: 0.2964327399392047
Weights: [0.11112758 0.00155855 0.1668604  0.714579   0.00587447]
Betas: [11  9 13  8  9]
Got-in: {False: 38, True: 62}
% Got-in: {False: 38.0, True: 62.0} %
--------------------------


In [934]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        alpha.append((i,k)) # ==> donne tous les alpha i k 

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [935]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}


In [936]:
#Clause 1
clause_1 = []

for i in range(nb_grades):
    for k in range(0,MAX_GRADE-1):
        for j in range(k+1,MAX_GRADE):
            clause_1.append([-v2i_alpha[(i,k)], v2i_alpha[(i,j)]])
         

In [937]:
#Clause 2
clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])



In [938]:
#Clause 3

clause_3 = []
for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 1:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(v2i_alpha[(i,student[i])])
        
            clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])

clause_3

[[137],
 [13, 136],
 [26, 135],
 [48, 134],
 [84, 133],
 [96, 132],
 [13, 26, 131],
 [13, 48, 130],
 [13, 84, 129],
 [13, 96, 128],
 [26, 48, 127],
 [26, 84, 126],
 [26, 96, 125],
 [48, 84, 124],
 [48, 96, 123],
 [84, 96, 122],
 [13, 26, 48, 121],
 [13, 26, 84, 120],
 [13, 26, 96, 119],
 [13, 48, 84, 118],
 [13, 48, 96, 117],
 [13, 84, 96, 116],
 [26, 48, 84, 115],
 [26, 48, 96, 114],
 [26, 84, 96, 113],
 [48, 84, 96, 112],
 [13, 26, 48, 84, 111],
 [13, 26, 48, 96, 110],
 [13, 26, 84, 96, 109],
 [13, 48, 84, 96, 108],
 [26, 48, 84, 96, 107],
 [13, 26, 48, 84, 96, 106],
 [137],
 [2, 136],
 [38, 135],
 [43, 134],
 [83, 133],
 [91, 132],
 [2, 38, 131],
 [2, 43, 130],
 [2, 83, 129],
 [2, 91, 128],
 [38, 43, 127],
 [38, 83, 126],
 [38, 91, 125],
 [43, 83, 124],
 [43, 91, 123],
 [83, 91, 122],
 [2, 38, 43, 121],
 [2, 38, 83, 120],
 [2, 38, 91, 119],
 [2, 43, 83, 118],
 [2, 43, 91, 117],
 [2, 83, 91, 116],
 [38, 43, 83, 115],
 [38, 43, 91, 114],
 [38, 83, 91, 113],
 [43, 83, 91, 112],
 [2, 38

In [939]:
clause_4 = []


for st_idx,student in enumerate(grades):
    if admissions[st_idx] == 0:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i])])
        
            clause_4.append(alpha+[-v2i_beta[frozenset(c)]])


In [940]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+1+A] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+A+1)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [941]:
#Lancer la résolution

myClauses= clause_1 + clause_2 + clause_3 + clause_4
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile.cnf")
res = exec_gophersat("./workingfile.cnf")

#Résultat
print(res)

(True, [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, -64, -65, -66, -67, -68, -69, -70, -71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, -106, -107, -108, -109, -110, -111, -112, -113, -114, -115, -116, -117, -118, -119, -120, -121, -122, -123, -124, -125, -126, -127, -128, -129, -130, -131, -132, -133, -134, -135, 136, 137], {(0, 0): False, (0, 1): False, (0, 2): False, (0, 3): False, (0, 4): False, (0, 5): False, (0, 6): False, (0, 7): False, (0, 8): False, (0, 9): False, (0, 10): False, (0, 11): False, (0, 12): False, (0, 13): False, (0, 14): False, (0, 15): False, (0, 16): False, (0, 17): False, (0, 18): False, (0, 19): False, (0, 20): False, (1, 0): True, (1, 1): True, 

In [942]:
_, variables_idx, d = res

In [943]:
predicted = []
for student in grades:
    validated_courses = set()
    for i,k in enumerate(student):
        if d[(i,k)]:
            validated_courses.add(i)
    validated_courses = frozenset(validated_courses)
    if d[validated_courses]:
        predicted.append(1)
    else:
        predicted.append(0)
predicted
        


[0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0]

In [944]:
from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(admissions,predicted))
print(f1_score(admissions,predicted))

1.0
1.0


SAT Solver Multi classes

In [945]:
import sys
sys.path.append('../')

from generator import GradesGenerator

In [946]:
size = 100
nb_grades = 5
noise = 0
nb_class = 1

s = {i for i in range(nb_grades)}

s = frozenset(s)

gen = GradesGenerator(size=size, nb_grades=nb_grades,noise=noise, nb_class=nb_class)

grades, admissions = gen.generate_grades()


if nb_class == 1:
    admissions = admissions.astype(int)


In [947]:
#Variable aplha
alpha = []
for i in range(nb_grades):
    for k in range(MAX_GRADE):
        for h in range(nb_class+1):
            alpha.append((i,k,h)) # ==> donne tous les alpha i k h

#Dictionnaire alpha
v2i_alpha = {v : i+1 for i,v in enumerate(alpha)} # à chaque variable associe un nombre
A = len(v2i_alpha)


In [948]:
from itertools import chain
from itertools import combinations

#Créer l'ensemble des subset 
def powerset(iterable): 
    s = list(iterable)
    return( chain.from_iterable(combinations(s, r) for r in range(len(s)+1)) )

#L'ensemble des subset 
subsets = list(powerset(s)) 


#Dictionnaire beta
v2i_beta = {frozenset(v) : A+i+1 for i,v in enumerate(subsets)}


In [949]:
clause_1 = []

for i in range(nb_grades):
    for k in range(MAX_GRADE-1):
        for h in range(nb_class):
            for j in range(k+1,MAX_GRADE):
                clause_1.append([-v2i_alpha[(i,k,h)], v2i_alpha[(i,j,h)]])

In [950]:
clause_2 = []

for i in subsets:
    C_prime = frozenset(i)
    for j in subsets:
        C = frozenset(j)
        if C.issubset(C_prime) and C != C_prime:
            clause_2.append([-v2i_beta[C], v2i_beta[C_prime]])


In [951]:
clause_3 = []


clause_3 = []
for st_idx,student in enumerate(grades):
    for c in subsets:
        alpha = []
        for i in c:
            alpha.append(v2i_alpha[(i,student[i],admissions[st_idx])])
    
        clause_3.append(alpha+[v2i_beta[frozenset(s.difference(c))]])





In [952]:
clause_4 = []

for st_idx,student in enumerate(grades):
    if admissions[st_idx] < nb_class:
        for c in subsets:
            alpha = []
            for i in c:
                alpha.append(-v2i_alpha[(i,student[i],admissions[st_idx]+1)])
        
            clause_4.append(alpha+[-v2i_beta[frozenset(c)]])


In [953]:
clause_5 = []

for k in range(MAX_GRADE):
    for i in range(nb_grades):
        for h in range(nb_class):
            for j in range(h+1, nb_class+1):
                clause_5.append([v2i_alpha[(i,k,h)],-v2i_alpha[(i,k,j)]])



In [954]:
#Construction du DIMCS et Résolution
i2v = {}

for i in range(len(v2i_alpha)):
    i2v[i+1] = list(v2i_alpha.keys())[list(v2i_alpha.values()).index(i+1)]

for i in range(len(v2i_beta)):
    i2v[i+A+1] = list(v2i_beta.keys())[list(v2i_beta.values()).index(i+1+A)]

import subprocess

def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./gophersat.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model if int(x) != 0], { i2v[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [955]:
myClauses= clause_1 + clause_2 + clause_3 + clause_4 + clause_5
myDimacs = clauses_to_dimacs(myClauses,len(i2v))

write_dimacs_file(myDimacs,"./workingfile_multi.cnf")
res = exec_gophersat("./workingfile_multi.cnf")

#Résultat

In [956]:
_, variables_idx, d = res

In [957]:
predicted = []
for student in grades:
    current_class = 0
    for h in range(nb_class+1):
        current_class = h
        validated_courses = set()
        for i,k in enumerate(student):
            if d[(i,k,h)]:
                validated_courses.add(i)
        validated_courses = frozenset(validated_courses)
        if validated_courses in d.keys():
            if d[validated_courses]:
                continue
            else:
                break
        else:
            break
    predicted.append(current_class-1)

predicted
        


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [958]:
from sklearn.metrics import f1_score, accuracy_score

print(accuracy_score(admissions,predicted))
print(f1_score(admissions,predicted, average='macro'))

0.54
0.35064935064935066
